In [1]:
import ipykernel
import os

In [2]:
%pwd

'e:\\ProjectPractice\\E2E-ML-Pipeline-MLOps-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
import pandas as pd
from dataclasses import dataclass
from pathlib import Path

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mobndash/E2E-ML-Pipeline-MLOps-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mobndash"
os.environ["MLFLOW_TRACKING_PASSWORD"]="8ba3f29223901e8f121aa24fe4e49db1d09adc15"

In [6]:
@dataclass(frozen=True)
class ModelEvaluationConfig :
    root_dir : Path
    test_data_path : Path
    model_path : str
    all_params : dict
    metric_file_name : Path
    mlflow_uri : str
    target_column : str

In [7]:
from src.ML_MLOps_MLFlow_Pipeline import *
from src.ML_MLOps_MLFlow_Pipeline.constants import *
from src.ML_MLOps_MLFlow_Pipeline.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
            
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig :
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            mlflow_uri = "https://dagshub.com/mobndash/E2E-ML-Pipeline-MLOps-MLFlow.mlflow",
            target_column = schema.name
        )
        
        return model_evaluation_config

In [9]:
import mlflow
import mlflow.sklearn
import pandas as pd
import joblib
import numpy as np
from pathlib import Path
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from src.ML_MLOps_MLFlow_Pipeline.utils.common import save_json# adjust import as needed

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_in_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
                
            if tracking_url_type_store != "file":
                local_model_path = Path("artifacts/model")
                mlflow.sklearn.save_model(model, path=local_model_path)
                mlflow.log_artifacts(local_model_path, artifact_path="model")
            else: 
                mlflow.sklearn.log_model(model, "model")


In [11]:
try :
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_in_mlflow()
except Exception as e :
    raise e

[2025-08-16 14:22:11,407: INFO: common: yaml file : <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-08-16 14:22:11,411: INFO: common: yaml file : <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-08-16 14:22:11,418: INFO: common: yaml file : <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-08-16 14:22:11,420: INFO: common: Created directory at path : artifacts]
[2025-08-16 14:22:11,423: INFO: common: Created directory at path : artifacts/model_evaluation]
[2025-08-16 14:22:11,912: INFO: common: Saved JSON file at path : artifacts\model_evaluation\metrics.json]


c:\Users\swapn\anaconda3\envs\mle2evenv\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
